In [ ]:
 #property
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
 #usphonebook
!pip install requests
!pip install beautifulsoup4

#property
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from pprint import pprint
options = webdriver.ChromeOptions()
options.add_argument('--log-level=3')
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(executable_path = r"chromedriver", chrome_options = options)

#usphonebook
import requests
from bs4 import BeautifulSoup

#general
import pandas as pd
import numpy as np


  Using cached urllib3-1.26.9-py2.py3-none-any.whl (138 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.11
    Uninstalling urllib3-1.25.11:
      Successfully uninstalled urllib3-1.25.11
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: use options instead of chrome_options


In [ ]:
#@title search_from_voter()
class search_from_voter():
  def __init__(self, person, voter):
    self.person = person
    self.voter = voter
  
  def find_rel_by_address_or_surname(self):
    relatives = np.array([])

    relative = self.voter[((self.voter.RESIDENTIAL_ADDRESS1 == self.person.RESIDENTIAL_ADDRESS1) | (self.voter.LAST_NAME == self.person.LAST_NAME) ) & (self.voter.SOS_VOTERID != self.person.SOS_VOTERID)].to_numpy()

    for i in range(relative.shape[0]):
      relatives = np.append(relatives, [self.person.SOS_VOTERID, relative[i][0]], axis=0)

    relatives = np.reshape(relatives, (-1, 2))
    return relatives


In [ ]:
#@title search_from_property
class search_from_property():
  def __init__(self, address):
    self.address = address
  
  def find_property_features(self):
    result_dict = {
        'names': [],
        'residential_class': []
    }
    #naviagate to housing website
    driver.get('https://myplace.cuyahogacounty.us/')

    #select the search by address button
    driver.find_element_by_id("Address").click()
    
    #find and enter address into search bar
    element = driver.find_element(By.ID,"txtData")
    element.send_keys(self.address)
    element.send_keys(Keys.ENTER)
    
    time.sleep(0.6)
    search_results = driver.find_element(By.XPATH, "//button[text()='Search Results']")
    if(search_results.is_displayed()):
        search_results.click()
    results = driver.find_element(By.ID, 'AddressInfo')
    text_elements = []
    for val in results.find_elements(By.CLASS_NAME, "backgroundzebra"):
        text_elements.append(val.text)
    number_of_results = int(len(text_elements)/6)

    for i in range(number_of_results):
        result_dict['names'].append(text_elements[i*6 + 1])
        parcel_btn = driver.find_element(By.ID, f"{i}")
        time.sleep(0.6)
        parcel_btn.click()
        time.sleep(0.6)
        driver.find_element(By.ID, f"btnGeneralInfo").click()
        time.sleep(0.6)
        tmp = driver.find_elements(By.CLASS_NAME, "dataBody")
        #print(tmp[0].text.splitlines())
        result_dict['residential_class'].append(tmp[0].text.splitlines()[10])
            
        driver.execute_script("window.history.go(-1)") 
    
    first_names, last_names = [], []
    for name in result_dict['names']:
      if(not name):
        first_names.append(None)
        last_names.append(None)
      else:
        first_name, last_name = filter_name(name)
        first_names.extend(first_name)
        last_names.extend(last_name)

    result_dict['first_name'] = first_names
    result_dict['last_name'] = last_names
    result_dict['residential_class'] = result_dict['residential_class'] * 2
    return result_dict

def filter_name(name):
  print(name)
  if(name.endswith(',')):
    name = name.rstrip(',')
  if('LLC'in name.upper()):
    return [None], [None]
  if(',' not in name):
    name = name.split(' ')[0] + ',' + " ".join(name.split(' ')[1:])
  if('&' in name):
    last_name_1 = name.split(',')[0]
    first_name_1 = name.split(',')[1].split('&')[0]

    if(len(name.split(',')) > 2):
      last_name_2 = name.split('&')[1].split(',')[0]
      first_name_2 = name.split('&')[1].split(',')[1]

    else:
      last_name_2 = last_name_1
      first_name_2 = name.split('&')[1]
      
    return [first_name_1.strip(), first_name_2.strip()], [last_name_1, last_name_2]

  elif(',' in name):
    first_name = name.split(',')[-1]
    last_name = name.split(',')[0]

  else:
    first_name = name.split(' ')[1]
    last_name = name.split(' ')[0]

  return [first_name], [last_name]


In [ ]:
#@title search_from_USPhoneBook
class search_from_USPhoneBook():
  def __init__(self, person):
    self.person = person

  def find_rel_by_usphonebook(self):
    relatives = self.find_relatives(self.person.FIRST_NAME, self.person.LAST_NAME, self.person.RESIDENTIAL_CITY, 'ohio', self.person.Age)
    return relatives

  def find_relatives(self, fname, lname, city, state, true_age):
    fname = str(fname).lower()
    lname = str(lname).lower()
    city = str(city).lower()
    state = str(state).lower()

    url = 'https://www.usphonebook.com/'
    
    URL = url + fname + '-' + lname + '/' + state + '/' + city
    header = {
          "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.82 Safari/537.360",
          'referrer': 'https://google.com'
          }
    page = requests.get(URL, headers=header)
    
    relatives = []

    if page.status_code != 200:
      relatives.append(['JOHN', 'DOE'])
      relatives = np.array(relatives)
      return relatives

    soup = BeautifulSoup(page.content, "html.parser")

    results = soup.find_all("div", {"class": "success-wrapper-block"})



    for res in results:
      # extract age
      person = str(res.find("span").next_sibling)
      age=""
      for m in person:
        if m.isdigit():
            age+=m

      # check if the age is correct
      if len(age) > 0:
        if int(age) == true_age:
          for relative in res.findAll("a", class_="ls_success-blue-link"):
            relatives.append(relative.find("span").text)
        if len(relatives) == 0:
          if int(age) == true_age-1:
            for relative in res.findAll("a", class_="ls_success-blue-link"):
              relatives.append(relative.find("span").text)
              
    # if no ages match, then pick the first result
    if len(relatives) == 0:
      for relative in results[0].findAll("a", class_="ls_success-blue-link"):
            relatives.append(relative.find("span").text)

    for i in range(len(relatives)):
      relatives[i] = relatives[i].split()
      for j in range(len(relatives[i])):
        relatives[i][j] = relatives[i][j].upper()
    
    if len(relatives) == 0:
      relatives.append(['JOHN', 'DOE'])

    relatives = np.array(relatives)
    
    return relatives

In [ ]:
def update_property_res(person, property_res):
  relatives_from_property = np.array([])
  for i in range(len(property_res['last_name'])):
    if isinstance(property_res['first_name'][i], type(None)):
      property_res['first_name'][i] = 'JOHN'
    if isinstance(property_res['last_name'][i], type(None)):
      property_res['last_name'][i] = 'DOE'

    if person.LAST_NAME == property_res['last_name'][i]:
      relatives_from_property = np.append(relatives_from_property, [property_res['first_name'][i].split()[0], property_res['last_name'][i]])
    elif property_res['residential_class'][i] == '5100 - 1-FAMILY PLATTED LOT':
      relatives_from_property = np.append(relatives_from_property, [property_res['first_name'][i].split()[0], property_res['last_name'][i]])
  relatives_from_property = np.reshape(relatives_from_property, (-1, 2))
  return relatives_from_property

In [ ]:
if __name__=="__main__":
  df_sample = pd.read_csv('sample_data.csv')
  df_voter = pd.read_csv('voter.csv')
  df_sample['RESIDENTIAL_CITY'] = df_sample.RESIDENTIAL_CITY.replace(' ', '-', regex=True)
  df_sample['LAST_NAME'] = df_sample.LAST_NAME.replace(' ', '-', regex=True)

  # n_people = df_sample.shape[0]
  n_people = 100
  res = np.array([])
  for i in range(n_people):
    i = i + 200
    person = df_sample.iloc[i, :]
    # from voter
    voter_searcher = search_from_voter(person, df_voter)
    voter_res = voter_searcher.find_rel_by_address_or_surname()

    # from property
    property_searcher = search_from_property(person.RESIDENTIAL_ADDRESS1)
    res_from_property = property_searcher.find_property_features()
    property_res = update_property_res(person, res_from_property)

    # from usphonebook
    usphonebook_searcher = search_from_USPhoneBook(person)
    usphonebook_res = usphonebook_searcher.find_rel_by_usphonebook()


    # prepare for inner join to check overlaps
    df_voter_res = df_voter[df_voter.SOS_VOTERID.isin(voter_res[:,1])]
    df_property_res = pd.DataFrame(property_res)
    df_property_res.columns=['FIRST_NAME', 'LAST_NAME']
    df_usphone_res = pd.DataFrame(usphonebook_res)
    df_usphone_res.columns=['FIRST_NAME', 'LAST_NAME']


    # voter join property
    vp_join = df_voter_res.merge(df_property_res, how='inner', on=['FIRST_NAME', 'LAST_NAME'])
    # voter join usphonebook
    vu_join = df_voter_res.merge(df_usphone_res, how='inner', on=['FIRST_NAME', 'LAST_NAME'])
    # property join usphonebook
    pu_join = df_property_res.merge(df_usphone_res, how='inner', on=['FIRST_NAME', 'LAST_NAME'])
    # all three
    vpu_join = df_voter_res.merge(df_property_res, how='inner', on=['FIRST_NAME', 'LAST_NAME']).merge(df_usphone_res, how='inner', on=['FIRST_NAME', 'LAST_NAME'])

    res = np.append(res, [person.FIRST_NAME, person.LAST_NAME, voter_res.shape[0], property_res.shape[0], usphonebook_res.shape[0], vp_join.shape[0], vu_join.shape[0], pu_join.shape[0], vpu_join.shape[0]], axis=0)
  # end for loop

  # reshape res arr
  res = np.reshape(res, (n_people, -1))

In [ ]:
df_res = pd.DataFrame(res)
df_res.columns = ['FIRST_NAME', 'LAST_NAME', 'from_voter', 'from_property', 'from_usphonebook', 'voter_join_property', 'voter_join_usphonebook', 'property_join_usphonebook', 'join_all']

# Outer Join

In [ ]:
def find_metrics(y_pred, y_true):
  tp = pd.merge(y_pred, y_true, how='inner', on=['SOS_VOTERID'])
  fp = y_pred[~y_pred.SOS_VOTERID.isin(tp.SOS_VOTERID)]
  fn = y_true[~y_true.SOS_VOTERID.isin(tp.SOS_VOTERID)]
  return tp.shape[0], fp.shape[0], fn.shape[0]

In [ ]:
#tp, fp, fn
parent_metrics = np.array([0, 0, 0])
child_metrics = np.array([0, 0, 0])
spouse_metrics = np.array([0, 0, 0])
sibling_metrics = np.array([0, 0, 0])

for i in range(100):
  person = df_sample.iloc[i, :]

  #search info
  voter_searcher = search_from_voter(person, df_voter)
  voter_res = voter_searcher.find_rel_by_address_or_surname()

  property_searcher = search_from_property(person.RESIDENTIAL_ADDRESS1)
  property_res = property_searcher.find_property_features()
  property_res = update_property_res(person, property_res)

  usphonebook_searcher = search_from_USPhoneBook(person)
  usphonebook_res = usphonebook_searcher.find_rel_by_usphonebook()

  #convert type
  df_voter_res = df_voter[df_voter.SOS_VOTERID.isin(voter_res[:,1])]

  df_property_res = pd.DataFrame(property_res)
  df_property_res.columns=['FIRST_NAME', 'LAST_NAME']

  df_usphone_res = pd.DataFrame(usphonebook_res)
  df_usphone_res.columns=['FIRST_NAME', 'LAST_NAME']

  #search for details from voting
  df_property_res_detail = df_voter.merge(df_property_res, how='inner', on=['FIRST_NAME', 'LAST_NAME'])
  relatives = pd.concat([df_voter_res, df_property_res_detail]).reset_index(drop=True)
  relatives_true = df_voter.merge(df_usphone_res, how='inner', on=['FIRST_NAME', 'LAST_NAME']).reset_index(drop=True)

  #rules
  #parent appropiate gender, age difference > 20 and < 60
  parent_pred = relatives[(relatives.Age >= person.Age + 20) & (relatives.Age <= person.Age + 60) & (relatives.Registration_age>0)]
  parent_true = relatives_true[(relatives_true.Age >= person.Age + 20) & (relatives_true.Age <= person.Age + 60)]

  #child age difference > 20
  child_pred = relatives[relatives.Age <= person.Age -20]
  child_true = relatives_true[relatives_true.Age <= person.Age -20]

  #spouse if opposite gender and age differnece < 20 and live in same address
  spouse_pred = relatives[(relatives.Gender != person.Gender) & (np.absolute(relatives.Age - person.Age) <= 20) & (relatives.RESIDENTIAL_ADDRESS1 == person.RESIDENTIAL_ADDRESS1)]
  spouse_true = relatives_true[(relatives_true.Gender != person.Gender) & (np.absolute(relatives_true.Age - person.Age) <= 20) & (relatives_true.RESIDENTIAL_ADDRESS1 == person.RESIDENTIAL_ADDRESS1)]

  #sibling if age differnece < 20 and live in different address
  sibling_pred = relatives[(np.absolute(relatives.Age - person.Age) <= 20) & (relatives.RESIDENTIAL_ADDRESS1 != person.RESIDENTIAL_ADDRESS1) & (relatives.Registration_age>0)]
  sibling_true = relatives_true[(np.absolute(relatives_true.Age - person.Age) <= 20) & (relatives_true.RESIDENTIAL_ADDRESS1 != person.RESIDENTIAL_ADDRESS1)]

  #calculate metrics
  parent_metrics += find_metrics(parent_pred, parent_true)
  child_metrics += find_metrics(child_pred, child_true)
  spouse_metrics += find_metrics(spouse_pred, spouse_true)
  sibling_metrics += find_metrics(sibling_pred, sibling_true)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  from ipykernel import kernelapp as app


In [ ]:
parent_metrics

array([  6, 187,  26])

In [ ]:
child_metrics

array([ 2, 20,  5])

In [ ]:
spouse_metrics

array([6, 2, 0])

In [ ]:
sibling_metrics

array([ 11, 466,  14])